In [1]:
import findspark
findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("argentinaDatosAdd")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

In [3]:
from pyspark.sql.functions import *

In [18]:
pas = 'p4ssW0rdP4r4Agr1cultur4'

In [4]:
df = spark.read.format("jdbc") \
.option("url", "jdbc:postgresql://postgres/agricultura") \
.option("driver", "org.postgresql.Driver") \
.option("dbtable", "agricultura.argentina_datos") \
.option("user", "agricultura") \
.option("password", pas) \
.load()

In [7]:
provRindeCultivos = (
df.groupBy('provincia', 'cultivo', 'campania', 'fecha')
.agg(avg('rendimiento_kgha').alias('rinde_kxha'))
.orderBy(desc('rinde_kxha'))
)

In [8]:
provRindeCultivos \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/agricultura") \
    .option("dbtable", "agricultura.prov_rinde") \
    .option("user", "agricultura") \
    .option("password", pas) \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [9]:
sojaBArinde = (
df.filter(df['provincia'] == "BUENOS AIRES")
.filter(df['cultivo'] == 'Soja total')
.select('provincia','rendimiento_kgha', 'campania', 'fecha')
)

In [10]:
sojaBArinde \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/agricultura") \
    .option("dbtable", "agricultura.soja_ba_rinde") \
    .option("user", "agricultura") \
    .option("password", pas) \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [5]:
df.show(2)

+------------+---------+---------------+------------+----------+--------+-----------+--------+-----------+------------+-------------+----------------+----------+
|id_provincia|provincia|id_departamento|departamento|id_cultivo| cultivo|id_campania|campania|ha_sembrada|ha_cosechada|produccion_tn|rendimiento_kgha|     fecha|
+------------+---------+---------------+------------+----------+--------+-----------+--------+-----------+------------+-------------+----------------+----------+
|          90|  TUCUMAN|             91|      SIMOCA|        34|Soja 1ra|         45| 2013/14|       3215|        3169|         6021|            1900|2013-01-01|
|          90|  TUCUMAN|             91|      SIMOCA|        34|Soja 1ra|         46| 2014/15|       3045|        2811|         5622|            2000|2014-01-01|
+------------+---------+---------------+------------+----------+--------+-----------+--------+-----------+------------+-------------+----------------+----------+
only showing top 2 rows



In [21]:
(
    df.select('cultivo').distinct().show(100)
)

AttributeError: 'NoneType' object has no attribute '_jvm'

In [16]:
cultivosProvincias = (
df.withColumn("tn_cosechadas",(col("ha_cosechada") * col('rendimiento_kgha'))/1000)
.groupBy('cultivo', 'provincia', 'fecha')
.agg(sum('tn_cosechadas').alias('tn_total'))
.orderBy(desc('tn_total'))
)

In [19]:
cultivosProvincias \
.write \
.format('jdbc') \
.option('url', 'jdbc:postgresql://postgres/agricultura') \
.option('dbtable', 'agricultura.cultivos_provincias') \
.option('user', 'agricultura') \
.option('password', pas) \
.option('driver', 'org.postgresql.Driver') \
.mode('overwrite') \
.save()

In [20]:
spark.stop()

ModuleNotFoundError: No module named 'dotenv'